# Download and Inspect the Collection

The dataset was created from the Chronicling America collection — over 21 million digitized newspaper pages (1756–1963) curated by the Library of Congress and NEH. They used 39,330 pages (1800–1920), representing 53 US states, to ensure wide geographic and temporal coverage.

Source: https://dl.acm.org/doi/pdf/10.1145/3626772.3657891

GitHub: https://github.com/DataScienceUIBK/ChroniclingAmericaQA?tab=readme-ov-file

In [ ]:
!pip install -r requirements.txt


In [ ]:
import os
os.makedirs("data", exist_ok=True)

!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/test.json?download=true" -o data/test.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/train.json?download=true" -o data/train.json
!curl -L "https://huggingface.co/datasets/Bhawna/ChroniclingAmericaQA/resolve/main/dev.json?download=true" -o data/validation.json

import json

files = ["data/train.json", "data/validation.json", "data/test.json"]

for path in files:
    print(f"\n===== {path} =====")
    try:
        with open(path, "r", encoding="utf-8") as f:
            # Read a few hundred characters to see what kind of JSON it is
            head = f.read(500)
            print("Preview of first 500 characters:\n")
            print(head[:500])
        # Try to load only part of the file
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            print(f"\nLoaded {len(data)} items (list).")
            print("Dictionary keys:", list(data[0].keys()))
            print(json.dumps(data[0], indent=2)[:600])
        elif isinstance(data, dict):
            print("\nTop-level is a dictionary. Keys:", list(data.keys()))
            for k, v in data.items():
                if isinstance(v, list):
                    print(f"Key '{k}' contains a list of {len(v)} items.")
                    if v:
                        print("First item keys:", list(v[0].keys()))
                        print(json.dumps(v[0], indent=2)[:600])
                        break
        else:
            print(f"Unexpected top-level type: {type(data)}")
    except Exception as e:
        print(f"Could not parse {path} as JSON: {e}")

# Create the Document Collection

To do that, we create a new json file that contains the 'para_id', 'context', 'raw_ocr', 'publication_date' keys, for all para_id in the collection.

para_id: is the id of a paragraph of a news paper page.

In [ ]:
import json
import os

inputs = ["data/train.json", "data/validation.json", "data/test.json"]
output = "data/document_collection.json"

def load_list_or_empty(path):
    if not os.path.exists(path) or os.path.getsize(path) == 0:
        print(f"Skipping {path} because it is missing or empty")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        if isinstance(data, list):
            return data
        print(f"Skipping {path} because it is not a list at the top level")
        return []
    except json.JSONDecodeError:
        print(f"Skipping {path} because it is not valid JSON")
        return []

def project(recs):
    out = []
    for r in recs:
        out.append({
            "para_id": r.get("para_id", ""),
            "context": r.get("context", ""),
            "raw_ocr": r.get("raw_ocr", ""),
            "publication_date": r.get("publication_date", "")
        })
    return out

all_recs = []
for p in inputs:
    recs = load_list_or_empty(p)
    print(f"Loaded {len(recs)} records from {p}")
    all_recs.extend(project(recs))

# deduplicate by para_id keeping the first one seen
uniq = {}
for rec in all_recs:
    pid = rec.get("para_id", "")
    if pid and pid not in uniq:
        uniq[pid] = rec

result = list(uniq.values())

with open(output, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"Wrote {len(result)} records to {output}")
print(json.dumps(result[:3], indent=2))

## You should check that the collection you have matches that of the paper!

In [ ]:
import pandas as pd
for path in inputs:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        df_check = pd.read_json(path)
        print(f'Shape of {path}: {df_check.shape}')

The dimensions match the ones of the paper at https://github.com/DataScienceUIBK/ChroniclingAmericaQA

# Create the Test Queries Data Structure

We keep the first 10.000 queries due to memory errors in the free colab version.

To be comparable, please keep the top 10.000 queries for evaluation.

In [ ]:
import json
import re
import unicodedata
import string

input_file = "data/test.json"
output_file = "data/test_queries.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

def clean_question(text):
    if not isinstance(text, str):
        return ""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(rf"[{re.escape(string.punctuation)}]", " ", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # collapse multiple spaces
    return text.strip()

# Extract and clean
queries = [
    {
        "query_id": item.get("query_id", ""),
        "question": clean_question(item.get("question", "")),
    }
    for item in data
]

# Sort by query_id (assuming numeric)
queries = sorted(queries, key=lambda x: int(x["query_id"]) if str(x["query_id"]).isdigit() else x["query_id"])

# Keep only the first 10,000
queries = queries[:10000]

# Save new JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(queries, f, ensure_ascii=False, indent=2)

print(f"Saved {len(queries)} entries to {output_file}")
print(json.dumps(queries[:3], indent=2))

# Create the Qrels for the test set

In [ ]:
input_file = "data/test.json"
qrels_file = "data/test_qrels.json"
answers_file = "data/test_query_answers.json"

# Load the data
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Build the qrels file: query_id, iteration=0, para_id, relevance=1
qrels = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1
    }
    for item in data
]

# Build the query_answers file: same plus answer and org_answer
query_answers = [
    {
        "query_id": item.get("query_id", ""),
        "iteration": 0,
        "para_id": item.get("para_id", ""),
        "relevance": 1,
        "answer": item.get("answer", ""),
        "org_answer": item.get("org_answer", "")
    }
    for item in data
]

# Save both files
with open(qrels_file, "w", encoding="utf-8") as f:
    json.dump(qrels, f, ensure_ascii=False, indent=2)

with open(answers_file, "w", encoding="utf-8") as f:
    json.dump(query_answers, f, ensure_ascii=False, indent=2)

print(f"Saved {len(qrels)} entries to {qrels_file}")
print(f"Saved {len(query_answers)} entries to {answers_file}")
print("Sample qrels entry:", qrels[0])
print("Sample query_answers entry:", query_answers[0])

# Retrieval

### Extract data from json files

In [ ]:
input_files = ['data/document_collection.json', 'data/test.json', 'data/test_qrels.json', 'data/test_queries.json', 'data/test_query_answers.json', 'data/train.json', 'data/validation.json']

dataframes = {}
for input_file in input_files:
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
        dataframes[input_file] = pd.read_json(input_file)

In [ ]:
dataframes['data/document_collection.json']

In [ ]:
dataframes['data/train.json']

**NOTE: in `data/document_collection.json` the rows are already deduplicated**

### _Preprocessing_

#### **Linguistic Processing**

##### Normalization
We lowercase everything and remove all special characters/tags

In [ ]:
import pandas as pd
import re
import unicodedata

def normalize_text(val):
    if not isinstance(val, str):
        return val
    val = unicodedata.normalize('NFKC', val)
    val = val.lower()
    val = re.sub(r'<[^>]+>', ' ', val)
    val = re.sub(r'[^a-z0-9\s]', '', val)
    val = re.sub(r'\s+', ' ', val).strip()
    return val

target_key = 'data/document_collection.json'

if target_key in dataframes:
    print(f"Normalizzazione in corso per: {target_key}...")
    
    # ESTRAI il dataframe
    df = dataframes[target_key]
    
    # APPLICA la normalizzazione a tutte le celle
    # Nota: .applymap() esegue la funzione su ogni singola cella del DataFrame
    df_normalized = df.applymap(normalize_text)
    
    # AGGIORNA il dizionario con la versione pulita
    dataframes[target_key] = df_normalized
    
    print(dataframes[target_key].head()) # Controllo rapido
else:
    print(f"Error: The key '{target_key}' does not exist in the dictionary dataframes.")

##### NER
We want to identify named-entities before lemmatizing the text, so that we do not lose any entity by "shrinking" words to their base forms.

In [ ]:
from transformers import AutoTokenizer, pipeline

MODEL_NAME = "impresso-project/ner-stacked-bert-multilingual-light"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ner_pipeline = pipeline(model=MODEL_NAME, tokenizer=tokenizer, trust_remote_code=True, device="cpu")

doc_collection = dataframes['data/document_collection.json']

test_subset = doc_collection.head(25).copy()

results = []
for index, row in test_subset.iterrows():
    sentence = str(row['context'])
    entities = ner_pipeline(sentence, tokens=sentence.split())
    results.append(entities)
test_subset['ner_entities'] = results

In [ ]:
test_subset[['context', 'ner_entities']]

##### Lemmatization
Placed here to standardize semantically the sentences in the documents

In [ ]:
import stanza
import pandas as pd

# 1. Configurazione Pipeline
# use_gpu=True se hai una GPU (Mac M1/M2 spesso richiede configurazioni extra, meglio False per stabilità ora)
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma', use_gpu=False, verbose=False)

# 2. Funzione Lemmatizzazione
def lemmatize_stanza(text):
    # Se non è una stringa o è vuota, ritorna il valore originale
    if not isinstance(text, str) or not text.strip():
        return text 
    
    doc = nlp(text)
    # Unisce i lemmi di tutte le parole in tutte le frasi
    lemmas = [word.lemma for sent in doc.sentences for word in sent.words]
    return " ".join(lemmas)

# 3. Esecuzione su TUTTO il DataFrame
target_key = 'data/document_collection.json'

if target_key in dataframes:
    print(f"Avvio lemmatizzazione totale su: {target_key}")
    df = dataframes[target_key]
    
    # Trova automaticamente tutte le colonne che contengono testo
    text_columns = df.select_dtypes(include=['object']).columns
    print(f"Colonne individuate: {list(text_columns)}")
    
    for col in text_columns:
        print(f"Processing colonna: '{col}'...")
        # Sovrascrive i dati con la versione lemmatizzata
        df[col] = df[col].apply(lemmatize_stanza)
    
    # Salva le modifiche
    dataframes[target_key] = df
    
    print("Fatto! Dataset completamente lemmatizzato.")
    print(df.head(3)) # Mostra le prime 3 righe per controllo
else:
    print(f"Errore: {target_key} non trovato.")

##### N-gram based tokenization
Important to place it after normalization, in this tokenization can be integrated a NER-aware part so that "the tokenization is also entity-guided"

### _Multi-field Indexing_

## Phase I

### **BM25 Retrieval from raw OCR (baseline 1)**

### **BM25 Retrieval from corrected OCR (baseline 2)**

### **BM25 Retrieval from both raw and corrected OCR using RRF formula (baseline 3)**